# [Installing the phyloseq package](https://joey711.github.io/phyloseq/install.html#quick_install)

https://vaulot.github.io/tutorials/Phyloseq_tutorial.html#gettin-started




https://bioconductor.org/packages/release/bioc/vignettes/phyloseq/inst/doc/phyloseq-analysis.html

The most updated examples are posted in our online tutorials from the [phyloseq home page](http://joey711.github.io/phyloseq/)



In [ ]:
# 5 Prerequisites to be installed
# R : https://pbil.univ-lyon1.fr/CRAN/

# R studio : https://www.rstudio.com/products/rstudio/download/#download

# Download this tutorial from GitHub : https://github.com/vaulot/R_tutorials/archive/master.zip

# Download and install the following libraries by running under R studio the following lines

# install.packages("dplyr")     # To manipulate dataframes
# install.packages("readxl")    # To read Excel files into R
# install.packages("ggplot2")   # for high quality graphics

# install.packages('BiocManager')
# library(BiocManager)
# # BiocManager::install() 
# BiocManager::install("phyloseq")
   

In [ ]:
library("readxl")       # necessary to import the data from Excel file
library("dplyr")        # filter and reformat data frames
library("phyloseq")
library("ggplot2")      # graphics
library('tidyr')
theme_set(theme_bw())
library('ape')

# [Importing phyloseq Data](http://joey711.github.io/phyloseq/import-data.html)

参考分析文档：

https://vaulot.github.io/tutorials/Phyloseq_tutorial.html#gettin-started

https://bioconductor.org/packages/release/bioc/vignettes/phyloseq/inst/doc/phyloseq-analysis.html

http://joey711.github.io/phyloseq/import-data.html

McMurdie and Holmes (2014) Shiny-phyloseq: Web Application for Interactive Microbiome Analysis with Provenance Tracking. Bioinformatics (Oxford, England) 31(2), 282–283.

McMurdie and Holmes (2013) phyloseq: An R package for reproducible interactive analysis and graphics of microbiome census data PLoS ONE 8(4):e61217


![](http://www.plosone.org/article/info:doi/10.1371/journal.pone.0061217.g003/largerimage)


## metagenome

写R函数来批量测试不同的疾病

In [1]:
library("phyloseq")


In [22]:
disease_project_physeq <- function(otutable,
                           taxtable,
                           sampledata,
                           treef = NULL, #nw format tree generated from ete3
                           outdir='./',
                           healthIDs=c('D006262'),
                           meshIDs=c('D003093', 'D006262')){
    meshID_allHealth_samples <- sampledata %>% filter(Disease.MESH.ID %in% meshIDs) 
    disease_projects <- meshID_allHealth_samples %>%
                        filter(!Disease.MESH.ID %in% healthIDs) %>% #projects with disease samples
                        .$Project.ID %>%
                        unique()
    if (!length(disease_projects)) return('disease has no control health samples')
    
    meshID_sample <- meshID_allHealth_samples %>%
                    filter(Project.ID %in% disease_projects)
    
    meshID_otutable <- otutable %>% 
        select(ncbi_taxon_id,as.vector(meshID_sample$Run.ID)) %>%
        filter(rowSums(.[-1])>0) 
    meshID_tax <- taxtable %>% filter(.$ncbi_taxon_id %in% meshID_otutable$ncbi_taxon_id)
    
    row.names(meshID_otutable) <- meshID_otutable$ncbi_taxon_id
    otu_mat <- meshID_otutable %>% select (-ncbi_taxon_id) 
    otu_mat <- otu_mat / 100 #relative abundance
    
    row.names(meshID_tax) <- meshID_tax$ncbi_taxon_id
    tax_mat <- meshID_tax %>% select (-ncbi_taxon_id) 
    
    row.names(meshID_sample) <- meshID_sample$Run.ID
    samples_df <- meshID_sample #%>% select (-Run.ID)
    
    OTU <- otu_table(as.matrix(otu_mat), taxa_are_rows = TRUE)
    TAX <- tax_table(as.matrix(tax_mat))  
    samples <- sample_data(samples_df)
    
    if (!is.null(treef)){
        tree <- read_tree(treef)
        pylsq <- phyloseq(OTU, TAX, samples, tree)
        
    }else{
        pylsq <- phyloseq(OTU, TAX, samples)
    }
    
    save(pylsq,file = paste0(outdir,'phyloseq.RData'))
    
    colnames(meshID_sample)[1] <- '#SampleID'
    write.table(meshID_sample,file=paste0(outdir,'sample_data.xls'),sep='\t',row.names = FALSE,quote = FALSE)
    
    otu_mat <- cbind(featureid = rownames(otu_mat), otu_mat)
    write.table(otu_mat,file=paste0(outdir,'otu_table.xls'),sep='\t', quote = FALSE, row.names = F)
    
    tax_mat <- cbind(featureid = rownames(tax_mat), tax_mat)
    write.table(tax_mat,file=paste0(outdir,'tax_table.xls'),sep='\t', quote = FALSE, row.names = F)
    
    write.tree(pylsq@phy_tree,file=paste0(outdir,'tree.nwk'))

    return(pylsq)

}

## Keep only samples with healthy controls to be analyzed



D006262	Health

D003093	Colitis, Ulcerative

In [251]:
diseasef='./meshID_phenotypes.csv'
disease = read.csv(diseasef,row.names = 1,header=T)
disease %>%head()
#疾病名字太长，还是用meshid
# metadata不修改了

,disease,term,note
,<fct>,<fct>,<fct>
0,C537163,Pediatric Autoimmune Neuropsychiatric Disorders Associated with Streptococcal infections,"OBSESSIVE-COMPULSIVE DISORDER and TIC DISORDERS that occur in children, typically under age 10, and is associated with group A beta-hemolytic Streptococcal infection. Onset occurs rapidly and patients may also exhibit emotional lability; SEPARATION ANXIETY DISORDER; ANOREXIA; and ADDH - like symptoms. AUTOIMMUNITY against proteins in the BASAL GANGLIA may cause this disorder."
1,D000067011,Severe Acute Malnutrition,"Acute form of MALNUTRITION which usually affects children, characterized by a very low weight for height (below -3z scores of the median World Health Organization standards), visible severe wasting, or occurrence of nutritional EDEMA. It can be a direct or indirect cause of fatality in children suffering from DIARRHEA and PNEUMONIA. Do not confuse with starvation, a condition in which the body is not getting enough food, usually for extended periods of time."
2,D000067877,Autism Spectrum Disorder,"Wide continuum of associated cognitive and neurobehavioral disorders, including, but not limited to, three core-defining features: impairments in socialization, impairments in verbal and nonverbal communication, and restricted and repetitive patterns of behaviors. (from DSM-V)"
3,D000236,Adenoma,A benign epithelial tumor with a glandular organization.
4,D000544,Alzheimer Disease,"A degenerative disease of the BRAIN characterized by the insidious onset of DEMENTIA. Impairment of MEMORY, judgment, attention span, and problem solving skills are followed by severe APRAXIAS and a global loss of cognitive abilities. The condition primarily occurs after age 60, and is marked pathologically by severe cortical atrophy and the triad of SENILE PLAQUES; NEUROFIBRILLARY TANGLES; and NEUROPIL THREADS. (From Adams et al., Principles of Neurology, 6th ed, pp1049-57)"
5,D000755,"Anemia, Sickle Cell","A disease characterized by chronic hemolytic anemia, episodic painful crises, and pathologic involvement of many organs. It is the clinical expression of homozygosity for hemoglobin S."


## metagenome




In [ ]:
sampledata <- read.csv('metagenome/Metagenome_metadata.xls',sep='\t')
sampledata <- unique(sampledata) #有重复行
diseaseIDs <- disease$disease[!disease$disease=='D006262' ]
genus_disease <- sampledata$Disease.MESH.ID[sampledata$Disease.MESH.ID %in% diseaseIDs & sampledata$Run.ID %in% colnames(gotutable)[-1]]
length(unique(genus_disease))
metagenome_diseaseIDs<- unique(genus_disease)

In [467]:
write.table(unique(genus_disease),'metagenome_otutable_diseaseIDs.tsv',quote = F,col.names = F)


In [4]:
genus_disease <- read.table('metagenome_otutable_diseaseIDs.tsv',row.names = 1)
metagenome_diseaseIDs<- unique(genus_disease$V2)

In [63]:
sampledata <- unique(sampledata) #有重复行

### metagenome genus

In [ ]:
# meta = read.table('./metagenome/diseases/UC/genus/otu_table.xls',sep='\t',header=T,row.names = 1)
# write.table(meta,file=paste0(outdir,'test.xls'),sep='\t', quote = FALSE, row.names = T)
# tax <- read.csv(file = 'metagenome/g_tax.txt',sep='\t',header = 0)
# tax_head <-  c("Domain", "Phylum", "Class", "Order", "Family", "Genus", "Species")
# taxtable <- separate(tax,V2,tax_head, sep=';')
# head(taxtable) #去掉了末尾的空s__

tax <- read.csv(file = 'metagenome/g_tax.csv')
tax_head <-  c("Domain", "Phylum", "Class", "Order", "Family", "Genus", "Species")
taxtable <- separate(tax,scientific_name,tax_head, sep=';')
otutable <- read.csv('metagenome/genus_table.csv')


In [73]:
sampledata <- sampledata[! sampledata$Run.ID %in% c('ERR2239214', 'ERR2239287', 'ERR2239800'),]
write.table(sampledata,'metagenome/Metagenome_metadata.uniq.xls',quote = F,col.names = T,row.names = F,sep='\t')

In [75]:
treef <- './metagenome/g_tree9.nwk'
healthIDs=c('D006262')
# diseaseID=c('D003093') 

for (diseaseID in metagenome_diseaseIDs) {
  
    outdir <- Sys.glob(paste0('./metagenome/diseases/',diseaseID,'_*/genus/'))
    meshIDs = append(diseaseID, healthIDs)
    uc_phyloseq <- disease_project_physeq(otutable,
                                          taxtable,
                                          sampledata,
                                          treef=treef,
                                          outdir=outdir,
                                          healthIDs=healthIDs,
                                          meshIDs=meshIDs) 
  
}

In [127]:
length(metagenome_diseaseIDs)

[1] 49

### metagenome species

In [129]:
head(sampledata)

Run.ID,Project.ID,Experiment.type,Nr..reads.sequenced,Country,Sex,Host.age,BMI,Recent.antibiotics.use,Disease.MESH.ID,Disease.name,QC.status
<fct>,<fct>,<fct>,<int>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<int>,<int>
ERR011089,PRJEB2054,Metagenomics,4190512,Denmark,Female,59,NA,,D006262,1,1
ERR011090,PRJEB2054,Metagenomics,11169067,Denmark,Female,59,NA,,D006262,1,1
ERR011091,PRJEB2054,Metagenomics,7928489,Denmark,Female,59,NA,,D006262,1,1
ERR011092,PRJEB2054,Metagenomics,4478452,Denmark,Male,69,NA,,D006262,1,1
ERR011093,PRJEB2054,Metagenomics,10857325,Denmark,Male,69,NA,,D006262,1,1
ERR011094,PRJEB2054,Metagenomics,9858261,Denmark,Male,69,NA,,D006262,1,1


In [79]:
tax <- read.csv(file = 'metagenome/s_tax.csv')
tax_head <-  c("Domain", "Phylum", "Class", "Order", "Family", "Genus", "Species")
taxtable <- separate(tax,scientific_name,tax_head, sep=';')
otutable <- read.csv('metagenome/species_table.csv')

In [137]:
treef <- './metagenome/s_tree9.nwk'
healthIDs=c('D006262')

for (diseaseID in metagenome_diseaseIDs) {
  
    outdir <- Sys.glob(paste0('./metagenome/diseases/',diseaseID,'_*/species/'))
    meshIDs = append(diseaseID, healthIDs)
    uc_phyloseq <- disease_project_physeq(otutable,
                                          taxtable,
                                          sampledata,
                                          treef=treef,
                                          outdir=outdir,
                                          healthIDs=healthIDs,
                                          meshIDs=meshIDs) 
  
}

# Amplicon

## Amplicon  genus

In [ ]:
agtax <- read.csv(file = 'amplicon/g_tax.xls',sep='\t')
agtax_head <-  c("Domain", "Phylum", "Class", "Order", "Family", "Genus", "Species")
agtaxtable <- separate(agtax,scientific_name,tax_head, sep=';')
agotutable <- read.csv('amplicon/genus_table.csv')

In [184]:
head(agtaxtable)

ncbi_taxon_id,Domain,Phylum,Class,Order,Family,Genus,Species
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
10,k__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Cellvibrionales,f__Cellvibrionaceae,g__Cellvibrio,s__
18,k__Bacteria,p__Proteobacteria,c__delta/epsilon subdivisions,o__Deltaproteobacteria,f__Desulfuromonadales,g__Desulfuromonadaceae,s__Pelobacter
20,k__Bacteria,p__Proteobacteria,c__Alphaproteobacteria,o__Caulobacterales,f__Caulobacteraceae,g__Phenylobacterium,s__
22,k__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Alteromonadales,f__Shewanellaceae,g__Shewanella,s__
53,k__Bacteria,p__Proteobacteria,c__delta/epsilon subdivisions,o__Deltaproteobacteria,f__Myxococcales,g__Nannocystineae,s__Nannocystaceae
75,k__Bacteria,p__Proteobacteria,c__Alphaproteobacteria,o__Caulobacterales,f__Caulobacteraceae,g__Caulobacter,s__


In [185]:
head(sampledata)

Run.ID,Project.ID,Experiment.type,Nr.reads.sequenced,Country,Sex,Host.age,BMI,Recent.antibiotics.use,Disease.MESH.ID,QC.status,Nr.associated.phenotypes
<fct>,<fct>,<fct>,<int>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<int>,<int>
DRR049319,PRJDB4360,Amplicon,12954,Japan,Female,104,NA,,D006262,1,1
DRR067938,PRJDB4998,Amplicon,36757,Japan,Female,29,19.56,,D006262,1,1
DRR068126,PRJDB4998,Amplicon,25643,Japan,Male,84,19.04,,D006262,1,1
DRR068128,PRJDB4998,Amplicon,24410,Japan,Male,78,22.41,,D006262,1,1
DRR092034,PRJDB4360,Amplicon,NA,Japan,,NA,NA,,D006262,1,1
DRR092042,PRJDB4360,Amplicon,NA,Japan,,NA,NA,,D006262,1,1


In [144]:
asampledata <- read.table('amplicon/user_selected_run_list_edit.txt',sep='\t',header=T)
adiseaseIDs <- disease$disease[!disease$disease=='D006262' ]
sampledata<- asampledata[!is.na(asampledata$Disease.MESH.ID),] # del NA rows
sampledata <- unique(sampledata) #有重复行
agenus_disease <- sampledata$Disease.MESH.ID[sampledata$Disease.MESH.ID %in% adiseaseIDs & (sampledata$Run.ID %in% colnames(agotutable)[-1])]
write.table(unique(agenus_disease),'Amplicon_otutable_diseaseIDs.tsv',quote = F,col.names = F)

In [204]:
dim(sampledata)

[1] 19179    12

In [156]:
amplicon_diseaseIDs<- unique(agenus_disease)

In [202]:
duplicate_rownames <- c('ERR1075808', 'ERR1089893', 'ERR1090018', 'ERR1090266', 'ERR1090428', 'ERR1160595', 'ERR1160700', 'ERR1249859', 'ERR1250525', 'ERR1389830', 'ERR2032593', 'ERR2032823', 'ERR2032859', 'ERR2056811', 'ERR2057225', 'ERR2091879')
# sampledata[sampledata$Run.ID %in% duplicate_rownames,]
sampledata <- sampledata[! sampledata$Run.ID %in% duplicate_rownames,] #不能确定到底属于哪个疾病or健康，都删去
write.table(sampledata,'amplicon/amplicon_metadata.uniq.xls',quote = F,col.names = T,row.names = F,sep='\t')

In [203]:
treef <- './amplicon/g_tree9.nwk'
healthIDs=c('D006262')

for (diseaseID in amplicon_diseaseIDs) {
  
    outdir <- Sys.glob(paste0('./amplicon/diseases/',diseaseID,'_*/genus/'))
    meshIDs = append(diseaseID, healthIDs)
    uc_phyloseq <- disease_project_physeq(agotutable,
                                          agtaxtable,
                                          sampledata,
                                          treef=treef,
                                          outdir=outdir,
                                          healthIDs=healthIDs,
                                          meshIDs=meshIDs) 
}

## Amplicon  species

In [ ]:
astax <- read.csv(file = 'amplicon/s_tax.xls',sep='\t')
astax_head <-  c("Domain", "Phylum", "Class", "Order", "Family", "Genus", "Species")
astaxtable <- separate(astax,scientific_name,tax_head, sep=';')
asotutable <- read.csv('amplicon/species_table.csv')

In [248]:
treef <- './amplicon/s_tree91.nwk'  #without sci_name then can be read in R
healthIDs=c('D006262')

for (diseaseID in amplicon_diseaseIDs) {
  
    outdir <- Sys.glob(paste0('./amplicon/diseases/',diseaseID,'_*/species/'))
    meshIDs = append(diseaseID, healthIDs)
    uc_phyloseq <- disease_project_physeq(asotutable,
                                          astaxtable,
                                          sampledata,
                                          treef=treef,
                                          outdir=outdir,
                                          healthIDs=healthIDs,
                                          meshIDs=meshIDs) 
}

## Keep only samples to be analyzed



目录下会生成r.data 上传shiny-phyloseq server交互分析。


In [288]:
# genus_tree <- read_tree('metagenome/genus_tree.nw')
# uc_tree_phyloseq = merge_phyloseq(uc_phyloseq, genus_tree)
# merge tree 会自动过滤不存在的node

In [154]:
uc_phyloseq

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 78 taxa and 1066 samples ]
sample_data() Sample Data:       [ 1066 samples by 12 sample variables ]
tax_table()   Taxonomy Table:    [ 78 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 78 tips and 37 internal nodes ]